In [1]:
import os
import random
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

sys.path.append("..")
import warnings

import wandb
from otc.metrics.metrics import effective_spread
from scipy.stats import wilcoxon
from tqdm.auto import tqdm


/pfs/data5/home/kit/stud/uloak/thesis/thesis/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# set here globally
EXCHANGE = "cboe"
MODELS = ["gbm", "classical"]  # ["gbm", "classical"]
SUBSET = "test"  # "all"
STRATEGY = "supervised"  # "transfer"


In [3]:
KEY = f"{EXCHANGE}_{STRATEGY}_{SUBSET}"
DATASET = f"fbv/thesis/{EXCHANGE}_{STRATEGY}_raw:latest"

os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

run = wandb.init(project="thesis", entity="fbv")

# load unscaled data
artifact = run.use_artifact(DATASET)  # type: ignore
data_dir = artifact.download()

# load results
result_dirs = []
for model in MODELS:
    results = f"fbv/thesis/{EXCHANGE}_{model}_{STRATEGY}_{SUBSET}:latest"
    artifact = run.use_artifact(results)  # type: ignore
    result_dir = artifact.download()
    result_dirs.append(result_dir)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karelze (fbv). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact cboe_supervised_raw:latest, 1937.19MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact cboe_gbm_supervised_test:latest, 81.87MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact cboe_classical_supervised_test:latest, 222.80MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0


In [4]:
# p. 35-38
COLUMNS = [
    "buy_sell",
    "EXPIRATION",
    "QUOTE_DATETIME",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "ask_ex",
    "bid_ex",
    "myn",
    "OPTION_TYPE",
    "issue_type",
]


if SUBSET == "all":
    train = pd.read_parquet(
        Path(data_dir, "train_set"), engine="fastparquet", columns=COLUMNS
    )
    val = pd.read_parquet(
        Path(data_dir, "val_set"), engine="fastparquet", columns=COLUMNS
    )
    test = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=COLUMNS
    )
    eval_data = pd.concat([train, val, test])
    del train, val, test

elif SUBSET == "test":
    eval_data = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=COLUMNS
    )


results = []
for i, model in tqdm(enumerate(MODELS)):
    result = pd.read_parquet(Path(result_dirs[i], "results"), engine="fastparquet")
    result.columns = pd.MultiIndex.from_product([[model], result.columns])
    results.append(result)

results_data = pd.concat(results, axis=1, names=MODELS)

assert len(eval_data) == len(results_data)

X_print = eval_data

del results


2it [00:01,  1.03it/s]


In [5]:
# FIXME: select a subset of results for testing.
results_data = results_data[
    [
        ("gbm", "gbm(classical)"),
        ("gbm", "gbm(classical-retraining)"),
        ("gbm", "gbm(classical-size)"),
        ("gbm", "gbm(classical-size-retraining)"),
        ("gbm", "gbm(ml)"),
        ("gbm", "gbm(ml-retraining)"),
        # ("gbm", "gbm(semi-classical)"),
        # ("gbm",'gbm(semi-classical-size)'),
        ("classical", "tick(ex)"),
        ("classical", "quote(ex)"),
        ("classical", "lr(ex)"),
        ("classical", "emo(ex)"),
        ("classical", "clnv(ex)"),
        (
            "classical",
            "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)",
        ),
    ]
]


In [6]:
results_data


gbm                                                \
         gbm(classical) gbm(classical-retraining) gbm(classical-size)   
index                                                                   
29930521              1                         1                   1   
29894717              1                        -1                   1   
29909660              1                         1                   1   
29939322             -1                        -1                  -1   
29883818             -1                        -1                   1   
...                 ...                       ...                 ...   
37155409              1                         1                  -1   
37155410              1                         1                   1   
37104659             -1                        -1                  -1   
37108365              1                         1                   1   
37155411              1                         1                   1   

                                                                   classical  \
         gbm(classical-size-retraining) gbm(ml) gbm(ml-retraining)  tick(ex)   
index                                                                          
29930521                              1       1                  1       1.0   
29894717                              1       1                  1      -1.0   
29909660                              1       1                  1      -1.0   
29939322                             -1      -1                 -1      -1.0   
29883818                              1       1                  1      -1.0   
...                                 ...     ...                ...       ...   
37155409                             -1      -1                 -1       1.0   
37155410                              1       1                  1       1.0   
37104659                             -1      -1                 -1      -1.0   
37108365                              1       1                  1      -1.0   
37155411                              1       1                  1      -1.0   

                                            \
         quote(ex) lr(ex) emo(ex) clnv(ex)   
index                                        
29930521      -1.0   -1.0    -1.0     -1.0   
29894717       1.0    1.0     1.0      1.0   
29909660      -1.0   -1.0    -1.0     -1.0   
29939322      -1.0   -1.0     1.0     -1.0   
29883818       1.0    1.0     1.0      1.0   
...            ...    ...     ...      ...   
37155409      -1.0   -1.0    -1.0     -1.0   
37155410       1.0    1.0     1.0      1.0   
37104659       1.0    1.0     1.0      1.0   
37108365      -1.0   -1.0    -1.0     -1.0   
37155411      -1.0   -1.0    -1.0     -1.0   

                                                                                        
         trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)  
index                                                                                   
29930521                                                1.0                             
29894717                                                1.0                             
29909660                                               -1.0                             
29939322                                               -1.0                             
29883818                                                1.0                             
...                                                     ...                             
37155409                                               -1.0                             
37155410                                                1.0                             
37104659                                               -1.0                             
37108365                                                1.0                             
37155411                                                1.0               

In [7]:
LUT = {
    "Trade_Size(ex)->Quote(Best)->Depth(Best)->Quote(Ex)->Depth(Ex)->Rev_Tick(All)": "\gls{GBM}",
    "(Ex)": " (Ex)",
    "(Best)": " (Best)",
    "(Classical)": " (Classical)",
    "(Classical-Size)": " (Classical, Size)",
    "Rev_": "Rev. ",
    "Trade_Size": "Trade Size",
    "Depth": "Depth",
    "->": " $\\to$ ",
    "Lr": "\gls{LR}",
    "Emo": "\gls{EMO}",
    "Clnv": "\gls{CLNV}",
    "OPTION_TYPE": "Option Type",
    "_": "$\_",
    "Gbm": "\gls{GBM}",
}

LUT_INDEX = {
    "OPTION_TYPE": "Option Type",
    "issue_type": "Security Type",
    "TRADE_SIZE_binned": "Trade Size",
    "year_binned": "Year",
    "ttm_binned": "Time to Maturity",
    "myn_binned": "Moneyness",
    "prox_q_binned": "Location to Quote",
    "all": "All trades",
}


def cell_str(x):
    x = x.title()
    for orig, sub in LUT.items():
        x = x.replace(orig, sub)
    # title-case everything
    return x


def highlight_max(s, props=""):
    return np.where(s == np.nanmax(s.values), props, "")


In [8]:
def set_tex_style(styler, caption, label, bold_axis=1):
    res = styler.set_caption(caption)

    res = (
        res.apply(highlight_max, props="font-weight:bold;", axis=bold_axis)
        .format(precision=4, decimal=".", thousands=",", escape=False, hyperlinks=None)
        .format_index(cell_str, axis=0)
        .format_index(cell_str, axis=1)
        .to_latex(
            f"{label}.tex",
            siunitx=True,
            position_float="centering",
            hrules=True,
            clines="skip-last;data",
            label="tab:" + label,
            caption=caption,
            convert_css=True,
        )
    )
    return res


In [9]:
classifiers = results_data.columns.tolist()
criterions = list(LUT_INDEX)


## Unclassified by method

In [10]:
results_data.head()


gbm                                                \
         gbm(classical) gbm(classical-retraining) gbm(classical-size)   
index                                                                   
29930521              1                         1                   1   
29894717              1                        -1                   1   
29909660              1                         1                   1   
29939322             -1                        -1                  -1   
29883818             -1                        -1                   1   

                                                                   classical  \
         gbm(classical-size-retraining) gbm(ml) gbm(ml-retraining)  tick(ex)   
index                                                                          
29930521                              1       1                  1       1.0   
29894717                              1       1                  1      -1.0   
29909660                              1       1                  1      -1.0   
29939322                             -1      -1                 -1      -1.0   
29883818                              1       1                  1      -1.0   

                                            \
         quote(ex) lr(ex) emo(ex) clnv(ex)   
index                                        
29930521      -1.0   -1.0    -1.0     -1.0   
29894717       1.0    1.0     1.0      1.0   
29909660      -1.0   -1.0    -1.0     -1.0   
29939322      -1.0   -1.0     1.0     -1.0   
29883818       1.0    1.0     1.0      1.0   

                                                                                        
         trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)  
index                                                                                   
29930521                                                1.0                             
29894717                                                1.0                             
29909660                                               -1.0                             
29939322                                               -1.0                             
29883818                                                1.0

In [11]:
unclassified = (
    (results_data[results_data == 0.0].count(axis=0) / len(results_data.index))
    .sort_values(ascending=False)
    .to_frame(name="unclassified")
)


In [12]:
unclassified.style.pipe(
    set_tex_style,
    caption=(f"{KEY}-unclassified-long", "{key}-unclassified-short"),
    label=f"{KEY.lower()}-unclassfied",
    bold_axis=0,
)
unclassified


unclassified
classical tick(ex)                                                0.095195
          quote(ex)                                               0.058323
          emo(ex)                                                 0.025770
          clnv(ex)                                                0.020722
          lr(ex)                                                  0.001731
          trade_size(ex)->quote(best)->depth(best)->quote...      0.000037
gbm       gbm(classical)                                          0.000000
          gbm(classical-retraining)                               0.000000
          gbm(classical-size)                                     0.000000
          gbm(classical-size-retraining)                          0.000000
          gbm(ml)                                                 0.000000
          gbm(ml-retraining)                                      0.000000

## Fill in unclassified

Unclassified are `0`.

In [13]:
rng = np.random.default_rng(42)

# replace 0 -> nan -> [-1,1]
results_data.replace(0, np.nan, inplace=True)
filler = pd.DataFrame(
    rng.choice(a=[-1, 1], size=results_data.shape),
    index=results_data.index,
    columns=results_data.columns,
)
results_data.fillna(filler, inplace=True)


### Robustness Checks

In [14]:
# prepare columns for printing
X_print["ttm"] = (
    X_print["EXPIRATION"].dt.to_period("M")
    - X_print["QUOTE_DATETIME"].dt.to_period("M")
).apply(lambda x: x.n)

X_print["year"] = X_print["QUOTE_DATETIME"].dt.year

bins_tradesize = [-1, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
X_print["TRADE_SIZE_binned"] = pd.cut(
    X_print["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

# p. 38
bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
X_print["year_binned"] = pd.cut(X_print["year"], bins_years, labels=year_labels)

# p. 37
bins_ttm = [-1, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "<= 1",
    "(1-2]",
    "(2-3]",
    "(3-6]",
    "(6-12]",
    "> 12",
]
X_print["ttm_binned"] = pd.cut(X_print["ttm"], bins_ttm, labels=ttm_labels)

# Security type
# see 3.0a-mb-explanatory-data-analysis.ipynb
X_print["issue_type"] = X_print["issue_type"].map(
    {
        "0": "Stock option",
        "A": "Index option",
        "7": "Others",
        "F": "Others",
        "%": "Others",
        " ": "Others",
    }
)

# Moneyness p. 38
bins_myn = [-1, 0.7, 0.9, 1.1, 1.3, np.inf]
myn_labels = [
    "<= 0.7",
    "(0.7-0.9]",
    "(0.9-1.1]",
    "(1.1-1.3]",
    "> 1.3",
]
X_print["myn_binned"] = pd.cut(X_print["myn"], bins_myn, labels=myn_labels)

# mid p. 31 + extra category for unknowns
ask = X_print["ask_ex"]
bid = X_print["bid_ex"]
trade_price = X_print["TRADE_PRICE"]

mid = np.where(ask >= bid, (ask + bid) * 0.5, np.nan)
half_spread = np.where(ask >= bid, (ask - bid) * 0.5, np.nan)

prox_quotes = np.where(
    trade_price == mid,
    0,  # at mid
    np.where(
        (bid < trade_price) & (trade_price < ask),
        1,  # inside
        np.where(
            (trade_price == bid) | (trade_price == ask),
            2,  # at quotes
            np.where((trade_price < bid) | (ask < trade_price), 3, 4),
        ),
    ),
)  # outside + unclassifiable

bins_prox = [-np.inf, 0, 1, 2, 3, 4]
prox_labels = [
    "at mid",
    "inside",
    "at quotes",
    "outside",
    "unknown",
]

X_print["prox_q_binned"] = pd.cut(prox_quotes, bins_prox, labels=prox_labels)
X_print["mid"] = mid

# clean up empty buckets, as it causes empty grouping in result set generation
X_print["year_binned"] = X_print["year_binned"].cat.remove_unused_categories()
X_print["myn_binned"] = X_print["myn_binned"].cat.remove_unused_categories()
X_print["ttm_binned"] = X_print["ttm_binned"].cat.remove_unused_categories()
X_print["prox_q_binned"] = X_print["prox_q_binned"].cat.remove_unused_categories()

X_print["all"] = "all"

X_print.drop(
    columns=[
        "EXPIRATION",
        "QUOTE_DATETIME",
        "TRADE_SIZE",
        "ttm",
        "myn",
        "ask_ex",
        "bid_ex",
        "year",
    ],
    inplace=True,
)


In [15]:
X_print = pd.concat([X_print, results_data], axis=1)


In [16]:
X_print.head().T


index,29930521,29894717,29909660,29939322,29883818
buy_sell,1,1,1,-1,1
TRADE_PRICE,0.02,0.27,0.2,0.12,1.85
OPTION_TYPE,C,P,C,C,C
issue_type,Others,Stock option,Stock option,Others,Stock option
TRADE_SIZE_binned,"(0,1]","(5,11]","(0,1]","(0,1]","(3,5]"
year_binned,2016,2016,2016,2016,2016
ttm_binned,<= 1,<= 1,(1-2],<= 1,(1-2]
myn_binned,(0.7-0.9],(0.9-1.1],(0.7-0.9],(0.9-1.1],(0.9-1.1]
prox_q_binned,at quotes,inside,at quotes,inside,at quotes
mid,0.025,0.225,0.35,0.135,1.725


## Accuracy Calculation

In [17]:
# FIXME: Find better approach
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

result_dfs = []

for criterion in tqdm(criterions):
    results = []
    for classifier in tqdm(classifiers):
        res = (
            X_print.groupby([criterion])[["buy_sell", classifier]]
            .apply(
                lambda x: accuracy_score(x["buy_sell"].astype("int8"), x[classifier])
            )
            .mul(100)
            .rename(classifier)
        )
        #         acc_tot = accuracy_score(
        #             X_print["buy_sell"].astype("int8"), X_print[classifier]
        #         )

        #         res.loc["all"] = acc_tot * 100

        res.index.name = LUT_INDEX.get(criterion)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1).T
    result_df.style.pipe(
        set_tex_style,
        caption=(f"long-tbd", "short-tbd"),
        label=f"{KEY.lower()}-{criterion.lower()}",
    )

    # store all result sets for later use
    result_dfs.append(result_df)


100%|██████████| 8/8 [01:41<00:00, 12.68s/it]


In [18]:
master = pd.concat(result_dfs, axis=1, keys=list(LUT_INDEX.values())).T


In [19]:
master


gbm                            \
                               gbm(classical) gbm(classical-retraining)   
Option Type       C                 69.929332                 70.670593   
                  P                 70.760558                 71.474293   
Security Type     Index option      68.338618                 68.973765   
                  Others            71.493258                 72.101682   
                  Stock option      69.979251                 70.766447   
Trade Size        (0,1]             68.092407                 68.905028   
                  (1,3]             69.205331                 69.934296   
                  (3,5]             69.687525                 70.384972   
                  (5,11]            70.951622                 71.718895   
                  >11               74.590981                 75.183524   
Year              2016              70.843835                 71.708100   
                  2017              70.162344                 70.853945   
Time to Maturity  <= 1              70.555488                 71.298291   
                  (1-2]             71.600389                 72.370927   
                  (2-3]             71.099750                 71.848948   
                  (3-6]             70.271222                 70.937580   
                  (6-12]            69.303696                 69.926256   
                  > 12              64.125740                 64.835765   
Moneyness         <= 0.7            71.619038                 72.438982   
                  (0.7-0.9]         71.104891                 72.092804   
                  (0.9-1.1]         70.844434                 71.564006   
                  (1.1-1.3]         63.309037                 63.552605   
                  > 1.3             63.025877                 63.060715   
Location to Quote at mid            62.321390                 62.850135   
                  inside            70.660281                 71.231693   
                  at quotes         73.650816                 75.250689   
                  outside           73.053067                 72.932460   
                  unknown           85.758294                 86.161137   
All trades        all               70.308753                 71.037449   

                                                    \
                               gbm(classical-size)   
Option Type       C                      74.756527   
                  P                      74.645476   
Security Type     Index option           72.520754   
                  Others                 75.251770   
                  Stock option           74.662893   
Trade Size        (0,1]                  72.620323   
                  (1,3]                  73.683006   
                  (3,5]                  74.403358   
                  (5,11]                 75.010330   
                  >11                    78.800083   
Year              2016                   75.275833   
                  2017                   74.549874   
Time to Maturity  <= 1                   74.287969   
                  (1-2]                  75.493369   
                  (2-3]                  75.899742   
                  (3-6]                  76.207334   
                  (6-12]                 74.826243   
                  > 12                   72.253504   
Moneyness         <= 0.7                 77.268103   
                  (0.7-0.9]              75.173820   
                  (0.9-1.1]              74.973394   
                  (1.1-1.3]              70.194063   
                  > 1.3                  69.954130   
Location to Quote at mid                 67.947788   
                  inside                 73.680356   
                  at quotes              83.772936   
                  outside                73.673329   
                  unknown                85.924171   
All trades        all                    74.705837   

                                         

In [20]:
master.style.pipe(
    set_tex_style,
    caption=("master-long", "master-short"),
    label=f"{KEY}-master",
    bold_axis=0,
)


## Effective Spread 💴

In [21]:
results = []

# calculate true rel effective spread but not aggregated, convert to %
es_true = effective_spread(X_print["buy_sell"], X_print["TRADE_PRICE"], X_print["mid"], mode="none")
eps_true = np.empty(es_true.shape)
np.divide(es_true, X_print["mid"], out=eps_true, where=X_print["mid"] != 0)

rel_true = effective_spread(X_print["buy_sell"], X_print["TRADE_PRICE"], X_print["mid"], mode="relative")
nom_true = effective_spread(X_print["buy_sell"], X_print["TRADE_PRICE"], X_print["mid"], mode="nominal")

for classifier in tqdm(classifiers):
    nom_pred = effective_spread(X_print[classifier], X_print["TRADE_PRICE"], X_print["mid"], mode="nominal")
    rel_pred = effective_spread(X_print[classifier], X_print["TRADE_PRICE"], X_print["mid"], mode="relative")

    # calculate pred rel effective spread but not aggregated convert to %
    es_pred = effective_spread(X_print[classifier], X_print["TRADE_PRICE"], X_print["mid"], mode="none")
    eps_pred = np.empty(es_pred.shape)
    np.divide(es_pred, X_print["mid"], out=eps_pred, where=X_print["mid"] != 0)

    # optional: quoted rel spread
    # (ask -bid) / ask

    wilcoxon_res  = wilcoxon(eps_pred, eps_true, nan_policy="omit", zero_method="zsplit")

    res = pd.Series(
            {
                "nom_pred": nom_pred * 100,
                "rel_pred": rel_pred * 100,
                "nom_true": nom_true * 100,
                "rel_true": rel_true * 100,
                'statistic':wilcoxon_res.statistic,
                'pvalue':wilcoxon_res.pvalue
            }, name=classifier
        )
    results.append(res)

results = pd.concat(results, axis=1)

100%|██████████| 12/12 [00:25<00:00,  2.16s/it]


In [22]:
results.T.style.format("{:.3f}")

## Diffs 🔄️

In [23]:
# classical baselines

base = master[
    [
        ("classical", "quote(ex)"),
        (
            "classical",
            "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)",
        ),
        (
            "classical",
            "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)",
        ),
    ]
]

# my ml models
revised = master[
    [("gbm", "gbm(classical)"), ("gbm", "gbm(classical-size)"), ("gbm", "gbm(ml)")]
]


In [24]:
def combine_results(revised: pd.DataFrame, base: pd.DataFrame) -> pd.DataFrame:
    """
    Generate print layout like in Grauer et al.

    https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290

    # see p. https://texdoc.org/serve/siunitx/0
    """
    # first, second layer of colum index
    c_1 = revised.columns.get_level_values(1)
    c_2 = ["nom"]
    midx = pd.MultiIndex.from_product([c_1, c_2])

    # copy data from revised add as (column, "nom")
    combo = pd.DataFrame(revised.values, index=revised.index, columns=midx)

    for i, mul_col in enumerate(combo.columns):

        combo[(mul_col[0], "pm")] = (combo[mul_col] - base.iloc[:, i]).round(2)
        combo.sort_index(axis=1, inplace=True)

    return combo


In [25]:
diff = combine_results(revised, base)

diff.style.to_latex(
    f"diff-{KEY}.tex",
    siunitx=True,
    position_float="centering",
    hrules=True,
    clines="skip-last;data",
    label=f"tab:diff-{KEY}",
    caption=(f"long-diff-{KEY}", f"short-diff-{KEY}"),
    convert_css=True,
)


In [26]:
diff


gbm(classical)        gbm(classical-size)  \
                                          nom     pm                 nom   
Option Type       C                 69.929332   9.08           74.756527   
                  P                 70.760558   9.80           74.645476   
Security Type     Index option      68.338618  18.02           72.520754   
                  Others            71.493258   6.22           75.251770   
                  Stock option      69.979251  10.01           74.662893   
Trade Size        (0,1]             68.092407  10.02           72.620323   
                  (1,3]             69.205331   7.87           73.683006   
                  (3,5]             69.687525   8.59           74.403358   
                  (5,11]            70.951622   9.44           75.010330   
                  >11               74.590981  10.77           78.800083   
Year              2016              70.843835  10.19           75.275833   
                  2017              70.162344   9.19           74.549874   
Time to Maturity  <= 1              70.555488   8.77           74.287969   
                  (1-2]             71.600389  10.12           75.493369   
                  (2-3]             71.099750  10.01           75.899742   
                  (3-6]             70.271222  10.15           76.207334   
                  (6-12]            69.303696  10.18           74.826243   
                  > 12              64.125740  11.62           72.253504   
Moneyness         <= 0.7            71.619038  16.74           77.268103   
                  (0.7-0.9]         71.104891  12.05           75.173820   
                  (0.9-1.1]         70.844434   8.44           74.973394   
                  (1.1-1.3]         63.309037   7.76           70.194063   
                  > 1.3             63.025877  10.67           69.954130   
Location to Quote at mid            62.321390  12.36           67.947788   
                  inside            70.660281   2.84           73.680356   
                  at quotes         73.650816  38.31           83.772936   
                  outside           73.053067   2.24           73.673329   
                  unknown           85.758294  36.71           85.924171   
All trades        all               70.308753   9.41           74.705837   

                                         gbm(ml)         
                                   pm        nom     pm  
Option Type       C              8.64  77.082873  10.97  
                  P              7.63  76.864945   9.85  
Security Type     Index option   4.84  76.430684   8.75  
                  Others         6.69  77.361457   8.80  
                  Stock option   9.09  76.871719  11.30  
Trade Size        (0,1]          8.55  75.332737  11.26  
                  (1,3]          8.32  76.167690  10.80  
                  (3,5]          8.09  76.599365  10.28  
                  (5,11]         8.65  77.095774  10.73  
                  >11            7.12  80.454838   8.78  
Year              2016          10.81  77.831739  13.37  
                  2017           7.46  76.751275   9.66  
Time to Maturity  <= 1           7.67  76.510357   9.89  
                  (1-2]          8.49  77.917123  10.92  
                  (2-3]          8.31  78.030617  10.44  
                  (3-6]          8.64  78.289240  10.72  
                  (6-12]         8.26  77.438567  10.87  
                  > 12          12.32  75.045868  15.11  
Moneyness         <= 0.7         7.84  79.809461  10.38  
                  (0.7-0.9]      8.09  77.717153  10.63  
                  (0.9-1.1]      7.96  77.064097  10.05  
                  (1.1-1.3]     10.25  72.987062  13.04  
                  > 1.3         11.21  74.292110  15.55  
Location to Quote at mid         7.71  70.502941  10.26  
                  inside         8.09  75.552631   9.96  
                  at quotes      8.98  87.797960  13.01  
                  outside        0.00  73.190903  

## Statistics 📊

In [27]:
COLUMNS = [
    "buy_sell",
    "EXPIRATION",
    "QUOTE_DATETIME",
    "TRADE_SIZE",
    "ask_size_ex",
    "bid_size_ex",
    "TRADE_PRICE",
    "myn",
    "ROOT",
    "STRK_PRC",
    "OPTION_TYPE",
    "EXPIRATION",
]


train = pd.read_parquet(
        Path(data_dir, "train_set"), engine="fastparquet", columns=COLUMNS
)
val = pd.read_parquet(
        Path(data_dir, "val_set"), engine="fastparquet", columns=COLUMNS
)
test = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=COLUMNS
)
df = pd.concat([train, val, test])

In [28]:
df["ttm"] = (df["EXPIRATION"] - df["QUOTE_DATETIME"]).dt.days
df["date"] = df["QUOTE_DATETIME"].dt.date

In [ ]:
# daily stats
stat_3 = df.groupby("date")[["ROOT"]].nunique().agg(["mean", "median", "std"])
stat_4 = df.groupby("date")[["ROOT"]].nunique().quantile([0.05, 0.5,  0.95])

df["key"] =df["OPTION_TYPE"].astype(str) + df["ROOT"].astype(str) + df["STRK_PRC"].astype(str) + df["EXPIRATION"].astype(str)

stat_7 = df.groupby("date")["key"].nunique().agg(["mean", "median", "std"])
stat_8 = df.groupby("date")["key"].nunique().quantile([0.05, 0.5,  0.95])

daily_stats = pd.concat([stat_3, stat_4], axis=0).T
daily_stats_2 = pd.concat([stat_7, stat_8], axis=0).T

In [ ]:
# non daily stats
rel_columns = ["TRADE_SIZE", "ttm", "myn"]
stat_1 = df[rel_columns].agg(["mean","median", "std"])
stat_2 = df[rel_columns].quantile([0.05, 0.5,  0.95])

other_stats = pd.concat([stat_1, stat_2], axis=0).T

In [ ]:
stats = pd.concat([daily_stats, daily_stats_2, other_stats], axis=0)

In [ ]:
stats